#### LDA TOPIC MODELING 

In [1]:
import pandas as pd
import numpy as np


import string
import re
from pprint import pprint

# NLTK 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english') #this depends on each language

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

__Loading data__

In [2]:
df=pd.read_csv('./dataset/abcnews-date-text.csv',sep=',')

In [3]:
df.shape[0]

1082168

In [4]:
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [5]:
data=df['headline_text'].values.tolist()

In [6]:
pprint(data[:1])

['aba decides against community broadcasting licence']


__Tokenize words and Clean-up text__

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['aba', 'decides', 'against', 'community', 'broadcasting', 'licence']]


__Bigrams & Trigrams__

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['aba', 'decides', 'against', 'community', 'broadcasting', 'licence']


__Remove Stopwords, Make Bigrams and Lemmatize__

In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

__Let’s call the functions in order__

In [10]:
#To run this code it is necessary to get the spacy moidel: "!python - m spacy download en", last depends on the used language 

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'es' model, keeping only tagger component (for efficiency)
# python3 -m spacy download es
nlp = spacy.load("en_core_web_sm",disable=['parser', 'ner'])  #spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['decide', 'community', 'broadcasting', 'licence']]


__Create the Dictionary and Corpus needed for Topic Modeling__

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [12]:
id2word[0]

'broadcasting'

In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('broadcasting', 1), ('community', 1), ('decide', 1), ('licence', 1)],
 [('act', 1),
  ('aware', 1),
  ('defamation', 1),
  ('fire', 1),
  ('must', 1),
  ('witness', 1)]]

__Building the Topic Model__

In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=4,
                                           alpha='auto',
                                           per_word_topics=True)

__View the topics in LDA model__

In [15]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.188*"attack" + 0.106*"court" + 0.080*"leave" + 0.074*"tell" + '
  '0.072*"business" + 0.070*"accuse" + 0.070*"lose" + 0.067*"give" + '
  '0.057*"risk" + 0.055*"drug"'),
 (1,
  '0.231*"man" + 0.192*"charge" + 0.130*"murder" + 0.082*"open" + '
  '0.079*"arrest" + 0.055*"aboriginal" + 0.052*"threat" + 0.041*"fine" + '
  '0.028*"soldier" + 0.024*"expect"'),
 (2,
  '0.255*"victim" + 0.187*"hold" + 0.175*"become" + 0.131*"fund" + '
  '0.034*"iraqi" + 0.030*"independent" + 0.029*"councillor" + 0.025*"contest" '
  '+ 0.023*"rebuild" + 0.012*"summit"'),
 (3,
  '0.422*"australian" + 0.082*"decision" + 0.079*"break" + 0.052*"welcome" + '
  '0.050*"consider" + 0.045*"meeting" + 0.042*"access" + 0.037*"affect" + '
  '0.035*"double" + 0.028*"insurance"'),
 (4,
  '0.211*"die" + 0.141*"warn" + 0.106*"work" + 0.094*"announce" + '
  '0.092*"young" + 0.092*"case" + 0.086*"final" + 0.030*"protester" + '
  '0.019*"phone" + 0.019*"korean"'),
 (5,
  '0.268*"fire" + 0.145*"back" + 0.130*"miss" + 0.0

__Compute Model Perplexity and Coherence Score__

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -26.928621793599653

Coherence Score:  0.2320959610820565


__Visualize the topics-keywords__

In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.242668  0.378928       1        1  11.063669
1     -0.197928 -0.231857       2        1  10.758386
8     -0.190446 -0.170364       3        1  10.113615
5     -0.082913 -0.017052       4        1   6.958416
4     -0.084271 -0.017524       5        1   6.753817
19    -0.050889 -0.008133       6        1   6.195823
17    -0.049755 -0.007867       7        1   5.763697
3     -0.058366 -0.009870       8        1   5.115157
11    -0.020331 -0.002274       9        1   4.935418
16     0.017677  0.002896      10        1   4.631126
10     0.045058  0.005622      11        1   4.426466
18     0.045917  0.005694      12        1   3.943594
15     0.026771  0.003896      13        1   3.881978
12     0.062306  0.007040      14        1   3.475614
2      0.090602  0.008950      15        1   3.051699
7      0.091852  0.009009      16        1   3.014275
13     0.088889  0.008827      17        1   2.608514
9      0.137700  0.011159      18        1   1.648174
14     0.167907  0.011996      19        1   1.237465
6      0.202889  0.010925      20        1   0.423101, topic_info=           Term           Freq          Total Category  logprob  loglift
194      police  114356.000000  114356.000000  Default  30.0000  30.0000
19   australian   73607.000000   73607.000000  Default  29.0000  29.0000
10         call   67806.000000   67806.000000  Default  28.0000  28.0000
224         man   84838.000000   84838.000000  Default  27.0000  27.0000
24          win   52110.000000   52110.000000  Default  26.0000  26.0000
..          ...            ...            ...      ...      ...      ...
339        show       0.167325       4.177090  Topic20 -11.3651   2.2479
440    industry       0.167305       4.176499  Topic20 -11.3652   2.2479
301         new       0.167323       4.178944  Topic20 -11.3651   2.2474
834    increase       0.167284       4.176185  Topic20 -11.3653   2.2479
771       fight       0.167285       4.176765  Topic20 -11.3653   2.2477

[828 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
251       2  0.999797  aboriginal
199       8  0.999413      access
177       1  0.999831      accuse
4        13  0.999685         act
18        8  0.999319      affect
...     ...       ...         ...
9        13  0.999300     witness
270      13  0.998248         woe
172       5  0.999825        work
278       5  0.999799       young
96        9  0.999750       youth

[190 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 9, 6, 5, 20, 18, 4, 12, 17, 11, 19, 16, 13, 3, 8, 14, 10, 15, 7])